## Introduction

We will continue to investigate the Chinook database and answer some business questions using SQL.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ('table', 'view');

-- this query return information on the tables and views of database

Done.
Done.


[]

In [3]:
%%sql
SELECT
    DISTINCT(name) genre_types
FROM genre;

-- practice query

Done.
Done.


[]

In [4]:
%%sql
WITH subquery AS
(
SELECT
    CASE
        WHEN name LIKE "A%" THEN "a_artists"
        WHEN name LIKE "B%" THEN "b_artists"
        WHEN name LIKE "C%" THEN "c_artists"
        WHEN name LIKE "D%" THEN "d_artists"
        WHEN name LIKE "E%" THEN "e_artists"
        WHEN name LIKE "F%" THEN "f_artists"
        WHEN name LIKE "G%" THEN "g_artists"
        ELSE "unsorted"
        END
        AS alph_artist
FROM artist
)

SELECT
    DISTINCT alph_artist,
    COUNT(alph_artist) count
FROM subquery
GROUP BY alph_artist;

-- practice query

Done.
Done.


[]

## Selecting albums to purchase

In [5]:
%%sql
WITH tracks_sold_usa AS
(
SELECT
    *
FROM invoice_line il
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
WHERE i.billing_country = "USA"
)

SELECT
    g.name genre,
    COUNT(ts.track_id) track_count,
    CAST(COUNT(ts.track_id) AS FLOAT) / (SELECT COUNT(*) FROM tracks_sold_usa) percentage
FROM tracks_sold_usa ts
INNER JOIN track t ON t.track_id = ts.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 3 DESC;

Done.


genre,track_count,percentage
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


In [6]:
%%sql
WITH tracks_sold_usa AS
(
SELECT
    *
FROM invoice_line il
INNER JOIN invoice i ON i.invoice_id = il.invoice_id
WHERE i.billing_country = "USA"
)

SELECT
    ar.name artist,
    a.title album_title,
    SUM(ts.quantity) track_sales
FROM tracks_sold_usa ts
INNER JOIN track t ON t.track_id = ts.track_id
INNER JOIN album a on a.album_id = t.album_id
INNER JOIN artist ar on a.artist_id = ar.artist_id
GROUP BY t.album_id
ORDER BY 3 DESC
LIMIT 3;

Done.


artist,album_title,track_sales
Jimi Hendrix,Are You Experienced?,28
Nirvana,From The Muddy Banks Of The Wishkah [live],28
The Doors,The Doors,27


We find that Rock is by far the most popular genre in the US by track sales, with over 50% of all sales generated by this genre. 

Upon further investigation, we find the the most frequently bought album in the USA are:
* Are You Experienced? by Jimi Hendrix
* From The Muddy Banks Of The Wishkah[live] by Nirvana
* The Doors by The Doors

## Analysing Employee Sales Performance

In [7]:
%%sql

WITH spending_per_cust AS
(
SELECT
    i.customer_id,
    (c.first_name||""||c.last_name) customer,
    c.support_rep_id,
    SUM(i.total) total_spending
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
GROUP BY 1
)

SELECT
    SUM(spc.total_spending) sales,
    e.employee_id,
    (e.first_name||" "||e.last_name) employee,
    e.title,
    e.hire_date,
    COUNT(spc.customer_id) account_count
FROM employee e
LEFT JOIN spending_per_cust spc ON spc.support_rep_id = e.employee_id
GROUP BY 2
ORDER BY 1 DESC;

Done.


sales,employee_id,employee,title,hire_date,account_count
1731.5099999999998,3,Jane Peacock,Sales Support Agent,2017-04-01 00:00:00,21
1584.0000000000002,4,Margaret Park,Sales Support Agent,2017-05-03 00:00:00,20
1393.92,5,Steve Johnson,Sales Support Agent,2017-10-17 00:00:00,18
None,1,Andrew Adams,General Manager,2016-08-14 00:00:00,0
None,2,Nancy Edwards,Sales Manager,2016-05-01 00:00:00,0
None,6,Michael Mitchell,IT Manager,2016-10-17 00:00:00,0
None,7,Robert King,IT Staff,2017-01-02 00:00:00,0
None,8,Laura Callahan,IT Staff,2017-03-04 00:00:00,0


From our analysis we see that Jane Peacock was the best sales agent with a sales figure of USD 1731.51, followed by Margaret Parker with USD 1584 in her total sales, then followed by Steve Johnson with sales of USD 1393.92.

We noticed that only the sales representatives have accounts under their name and hence the rest of the staff does not have sales figures assigned to them.

Upon further investigation, we found that the earlier the hire date for an employee, the more accounts they are assigned, which contributes to higher sales. This is not suprising, however, the direct correlation made it seem as if the difference in sales value was more contributed to by an employee's tenure instead of sales ability.

## Analyzing Sales by Country

Now we will look to find the following figures for each country:
* total number of customers
* total value of sales
* average value of sales per customer
* average order value

Though we will be grouping countrys with singular customers into the 'Other' category.

In [8]:
%%sql
WITH invoice_customer_country AS
(
    SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
        END AS country,
        i.*
    FROM invoice i
    LEFT JOIN customer c ON i.customer_id = c.customer_id
)

SELECT
    country,
    SUM(total) total_sales,
    COUNT(DISTINCT customer_id) customer_count,
    SUM(total) / COUNT(DISTINCT customer_id) avg_sales_per_customer,
    SUM(total) / COUNT(*) avg_order_value
FROM
    (
    SELECT
        *,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM invoice_customer_country
    )
GROUP BY 1
ORDER BY sort ASC, 2 DESC;

Done.


country,total_sales,customer_count,avg_sales_per_customer,avg_order_value
USA,1040.4899999999998,13,80.0376923076923,7.942671755725189
Canada,535.5900000000001,8,66.94875000000002,7.047236842105265
Brazil,427.68000000000006,5,85.53600000000002,7.0111475409836075
France,389.0699999999999,5,77.81399999999998,7.781399999999998
Germany,334.62,4,83.655,8.161463414634147
Czech Republic,273.24000000000007,2,136.62000000000003,9.108000000000002
United Kingdom,245.52,3,81.84,8.768571428571429
Portugal,185.13000000000002,2,92.56500000000001,6.383793103448276
India,183.14999999999998,2,91.57499999999999,8.72142857142857
Other,1094.9400000000003,15,72.99600000000002,7.44857142857143


We can see that USA is the highest sales generating country and the largest market by customer count, therefore an extremely important market for the firm. Further resources should be invested in gaining deeper understanding the client base to increase sales.

On the other hand, Czech Republic is the country with the highest value accounts, with average sales per customer at USD 136.62, which is way above the average of the other countries/regions. There are only two customers who are in this country, hence it will be smart to tailor a premium scheme to retain these high spending account.

Canada, whilst having the second largest customer base, have the lowest sales per customer at USD 66.95. This suggest that we can target the market with discount or package schemes, which appeals to more price sensitive markets.

## Albums vs Individual Tracks

Currently, the store  only allow the customers to make purchases in one of two ways:
* purchase a whole album
* purchase a collection of one or more individual tracks

However, management is considering to change purchasing strategy to improve cost efficiency. They are considering to purchase only the most popluar tracks from each album from record companies, instead of purchasing every track from an album.

To find whether this change in purchasing strategy is to be implemented, we have to find the percentage of purchases are individual vs whole album to understand effect on overall revenue.

There are two 'edge cases' in this instance that we should look into. They are:
* albums that only one or two tracks, hence likely to be purchased by customers as part of a collection of individual tracks
* in order to get the few singles on top an album in a single purchase, customers might opt to manually select the album and singles together

But since we are looking to maximise the revenue, we can ignore the first case, and company data shows that the second case does not happen often enough to effect results, and could be ignored as well.

We will answer this question by finding the percentage of invoices that are of a single album only.

In [9]:
%%sql
WITH invoice_album_track AS
(
    SELECT
        ila.invoice_id,
        CASE
            WHEN (
                SELECT count(DISTINCT t.album_id)
                FROM invoice_line il
                LEFT JOIN track t ON il.track_id = t.track_id
                where il.invoice_id = ila.invoice_id
                ) = 1 THEN "album"
            ELSE "tracks"
        END AS album_or_track
    FROM invoice_line ila
    LEFT JOIN track ta ON ila.track_id = ta.track_id
    GROUP BY 1
)

SELECT
    album_or_track,
    COUNT(album_or_track) invoice_count,
    ROUND(CAST((COUNT(album_or_track)*100) AS FLOAT) / (SELECT COUNT(*) FROM invoice_album_track),2) percentage
FROM invoice_album_track
GROUP BY 1;

Done.


album_or_track,invoice_count,percentage
album,171,27.85
tracks,443,72.15


From the results we see that 72.2% of purchases were of selection of tracks instead of single albums, hence we would recommend changing the store's purchasing strategy from full albums to individual tracks as it seems to be more likely to eliminate unnecessary costs.

Alternative codes for this section.
```
%%sql

WITH invoice_first_track AS
    (
     SELECT
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    )

SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) percent
FROM
    (
    SELECT
        ifs.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id FROM invoice_line il2
                  WHERE il2.invoice_id = ifs.invoice_id

                  EXCEPT 

                  SELECT t.track_id FROM track t
                  WHERE t.album_id = (
                                      SELECT t2.album_id FROM track t2
                                      WHERE t2.track_id = ifs.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_first_track ifs
    )
GROUP BY album_purchase;
```